In [1]:
import sys, os
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from keras.models import load_model

Using TensorFlow backend.


In [0]:
num_features = 64
num_labels = 4
batch_size = 64
epochs = 100
width, height = 48, 48

In [0]:
#Mount your google drive
#from google.colab import drive
#drive.mount('/content/drive')
#emotion_dict = {0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy", 4: "Sad", 5: "Surprise", 6: "Neutral"}
#Following are labels
#Calm 6
#Surprise & Fear 5 & 2
#Anger 0


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#https://drive.google.com/open?id=1OnveSEG0q5CwEQeZOW3QotcL_GK4UZ2l
#/content/drive/My Drive/dataset/fer2013.csv
root_path = '/content/drive/My Drive/dataset/fer2013.csv'

In [0]:
data = pd.read_csv(root_path)

In [9]:
data.tail()

,emotion,pixels,Usage
35882,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest
35883,3,178 174 172 173 181 188 191 194 196 199 200 20...,PrivateTest
35884,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest
35885,3,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,PrivateTest
35886,2,19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...,PrivateTest


In [0]:

#Remove parts of the data from the dataset to make small part training.
indexNames_1 = data[ data['emotion'] == 1].index
indexNames_3 = data[ data['emotion'] == 3].index
indexNames_4 = data[ data['emotion'] == 4].index
 
data.drop(indexNames_1 , inplace=True)
#data.drop(indexNames_3 , inplace=True)
#data.drop(indexNames_4 , inplace=True)

In [0]:
data.drop(indexNames_3 , inplace=True)

In [0]:
data.drop(indexNames_4 , inplace=True)

In [13]:
data.groupby('emotion').size()

emotion
0    4953
2    5121
5    4002
6    6198
dtype: int64

In [0]:
#update 5 with 2
#data.loc[data['emotion'] == 5, 'emotion'] = 2

print(data['emotion']==1)

In [14]:
pixels = data['pixels'].tolist() # 1

faces = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')] # 2
    if (len(face)) < 2304:
      print("array length less than 2304")
      continue
    face = np.asarray(face).reshape(width, height) # 3
    
    # There is an issue for normalizing images. Just comment out 4 and 5 lines until when I found the solution.
    #face = face / 255.0 # 4
    #face = cv2.resize(face.astype('uint8'), (width, height)) # 5
    #face = face / 255.0
    faces.append(face.astype('float32'))

faces = np.asarray(faces)
faces = np.expand_dims(faces, -1) # 6

emotions = pd.get_dummies(data['emotion']).as_matrix() # 7
print( emotions )

[[1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 ...
 [0 0 0 1]
 [1 0 0 0]
 [0 1 0 0]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [15]:
X_train, X_test, y_train, y_test = train_test_split(faces, emotions, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=41)

print(y_train)


[[0 0 0 1]
 [0 0 0 1]
 [0 1 0 0]
 ...
 [0 0 0 1]
 [1 0 0 0]
 [1 0 0 0]]


In [18]:
print(X_train)

[[[[ 32.]
   [ 32.]
   [ 34.]
   ...
   [ 24.]
   [ 18.]
   [ 19.]]

  [[ 34.]
   [ 32.]
   [ 32.]
   ...
   [ 24.]
   [ 18.]
   [ 19.]]

  [[ 32.]
   [ 32.]
   [ 33.]
   ...
   [ 23.]
   [ 26.]
   [ 24.]]

  ...

  [[237.]
   [243.]
   [246.]
   ...
   [199.]
   [192.]
   [183.]]

  [[236.]
   [241.]
   [245.]
   ...
   [201.]
   [197.]
   [188.]]

  [[237.]
   [239.]
   [245.]
   ...
   [198.]
   [197.]
   [191.]]]


 [[[ 71.]
   [ 70.]
   [ 72.]
   ...
   [103.]
   [107.]
   [106.]]

  [[ 69.]
   [ 70.]
   [ 71.]
   ...
   [111.]
   [108.]
   [101.]]

  [[ 68.]
   [ 67.]
   [ 87.]
   ...
   [112.]
   [106.]
   [ 94.]]

  ...

  [[ 68.]
   [ 63.]
   [ 63.]
   ...
   [ 58.]
   [ 54.]
   [ 53.]]

  [[ 64.]
   [ 64.]
   [ 64.]
   ...
   [ 56.]
   [ 53.]
   [ 56.]]

  [[ 64.]
   [ 65.]
   [ 62.]
   ...
   [ 56.]
   [ 56.]
   [ 58.]]]


 [[[219.]
   [215.]
   [207.]
   ...
   [  6.]
   [  8.]
   [ 13.]]

  [[215.]
   [206.]
   [194.]
   ...
   [  5.]
   [  9.]
   [ 14.]]

  [[209.]
   [19

In [0]:
model = Sequential()

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1), data_format='channels_last', kernel_regularizer=l2(0.01)))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(2*2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*num_features, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels, activation='softmax'))

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 46, 46, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 23, 23, 128)       73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 23, 23, 128)      

In [0]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])

In [0]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3, verbose=1)

In [0]:
tensorboard = TensorBoard(log_dir='/tmp')

In [0]:
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=8, verbose=1, mode='auto')

In [0]:
MODELPATH = '/tmp/model.h5'
checkpointer = ModelCheckpoint(MODELPATH, monitor='val_loss', verbose=1, save_best_only=True)

In [27]:
model.fit(np.array(X_train), np.array(y_train),
          batch_size=batch_size,
          epochs=60,
          verbose=1,
          validation_data=(np.array(X_test), np.array(y_test)),
          shuffle=True,
          callbacks=[lr_reducer, tensorboard, checkpointer])
#callbacks=[lr_reducer, tensorboard, early_stopper, checkpointer]

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 16421 samples, validate on 2028 samples


Epoch 1/60
16421/16421 [==============================] - 36s 2ms/step - loss: 1.6518 - acc: 0.2774 - val_loss: 1.3783 - val_acc: 0.2890

Epoch 00001: val_loss improved from inf to 1.37832, saving model to /tmp/model.h5
Epoch 2/60
16421/16421 [==============================] - 32s 2ms/step - loss: 1.4014 - acc: 0.2988 - val_loss: 1.3895 - val_acc: 0.2899

Epoch 00002: val_loss did not improve from 1.37832
Epoch 3/60
16421/16421 [==============================] - 32s 2ms/step - loss: 1.3684 - acc: 0.3274 - val_loss: 1.3220 - val_acc: 0.3664

Epoch 00003: val_loss improved from 1.37832 to 1.32201, saving model to /tmp/model.h5
Epoch 4/60
16421/16421 [==============================] - 32s 2ms/step - loss: 1.3107 - acc: 0.3838 - val_loss: 1.4496 - val_acc: 0.2954

Epoch 00004: val_loss did not improve from 1.32201
Epoch 5/60
16421/16421 [=========

In [28]:
scores = model.evaluate(np.array(X_test), np.array(y_test), batch_size=batch_size)
print("Loss: " + str(scores[0]))
print("Accuracy: " + str(scores[1]))

2028/2028 [==============================] - 1s 628us/step
Loss: 1.3247169223292574
Accuracy: 0.6844181456039173
